# Experimentos: Baseline com Rede Neural Clássica(MLP)

Este notebook estabelece baselines de performance com um modelo de Rede Neural Clássica(MLP) em datasets

# Dataset = Breast Cancer Wisconsin (Diagnostic)

In [1]:
from sklearn.datasets import load_breast_cancer 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier  # Importando MLP
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd

## Loading data and initial analysis

In [2]:
X, y = load_breast_cancer(return_X_y=True)  # Carregando o dataset

# Verificando as dimensões dos nossos dados
print(f"Dimensões dos dados: {X.shape}")


# Vendo a distribuição das classes (0 e 1)
print("\nDistribuição das classes:")
# Usando numpy para contar as ocorrências de cada classe
unique, counts = np.unique(y, return_counts=True)
for class_label, count in zip(unique, counts):
    print(f"Classe {class_label}: {count} ocorrências")


Dimensões dos dados: (569, 30)

Distribuição das classes:
Classe 0: 212 ocorrências
Classe 1: 357 ocorrências


## Split data into training and testing sets

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y) # Stratify -> mantém a proporção das classes no split

print(f"Dimensões do conjunto de treino: {X_train.shape}")
print(f"Dimensões do conjunto de teste: {X_test.shape}")


Dimensões do conjunto de treino: (426, 30)
Dimensões do conjunto de teste: (143, 30)


## Normalize data

In [4]:
# normalizing the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)  # Ajusta o scaler e transforma os dados de treino
X_test = scaler.transform(X_test)  # Transforma os dados de teste com o mesmo scaler

print("\nDados normalizados")






Dados normalizados


## applying pca

In [5]:
from sklearn.decomposition import PCA

N_COMPONENTES_BC = 5
pca_bc = PCA(n_components=N_COMPONENTES_BC)


X_train_pca = pca_bc.fit_transform(X_train)
X_test_pca = pca_bc.transform(X_test)

print(f"PCA aplicado. Dimensões reduzidas para {N_COMPONENTES_BC} componentes.")

PCA aplicado. Dimensões reduzidas para 5 componentes.


# Training MLP model

In [6]:
# Treinando o modelo Rede neural MLP
mlp_breast = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
mlp_breast.fit(X_train_pca, y_train)  # Treinando o modelo com os dados de treino
print("\nMLP com PCA treinado com sucesso!")


MLP com PCA treinado com sucesso!


# Predictions

In [7]:
# predicting the test set com MLP
mlp_predictions_breast = mlp_breast.predict(X_test_pca)
# Avaliando o modelo
print("\nRelatório de Classificação PCA 5 features:") 
print(classification_report(y_test, mlp_predictions_breast, target_names=load_breast_cancer().target_names))


Relatório de Classificação PCA 5 features:
              precision    recall  f1-score   support

   malignant       0.93      0.96      0.94        53
      benign       0.98      0.96      0.97        90

    accuracy                           0.96       143
   macro avg       0.95      0.96      0.96       143
weighted avg       0.96      0.96      0.96       143



# Dataset = Student Performance

In [8]:
# Importando o dataset
student_performance = 'student-por.csv'

df_student_performance = pd.read_csv(student_performance, sep=',')  # Lendo o dataset com separador ','

# 3. Replicamos a separação que a biblioteca fazia para nós:
# y são as colunas de notas
y = df_student_performance[['G1', 'G2', 'G3']]
# X são TODAS as outras colunas
X = df_student_performance.drop(columns=['G1', 'G2', 'G3'])

# --- Verificações para confirmar que deu tudo certo ---
print("--- Dataset carregado localmente com sucesso! ---")
print("Formato das features (X):", X.shape)
print("Formato dos alvos (y):", y.shape)

print("\n5 primeiras linhas das features (X):")


--- Dataset carregado localmente com sucesso! ---
Formato das features (X): (649, 30)
Formato dos alvos (y): (649, 3)

5 primeiras linhas das features (X):


In [9]:
X.head(5) 

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,yes,no,no,4,3,4,1,1,3,4
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,yes,no,5,3,3,1,1,3,2
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,yes,no,4,3,2,2,3,3,6
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,yes,3,2,2,1,1,5,0
4,GP,F,16,U,GT3,T,3,3,other,other,...,yes,no,no,4,3,2,1,2,5,0


In [10]:
# Dimensões dos dados do Student Performance Dataset
print("\nDimensões do Student Performance Dataset:")
print(f"X: {X.shape}")
print(f"y: {y.shape}")



Dimensões do Student Performance Dataset:
X: (649, 30)
y: (649, 3)


In [11]:
# Data cleaning
# check for missing values
print("\nVerificando valores ausentes:")
print(X.isnull().sum())


Verificando valores ausentes:
school        0
sex           0
age           0
address       0
famsize       0
Pstatus       0
Medu          0
Fedu          0
Mjob          0
Fjob          0
reason        0
guardian      0
traveltime    0
studytime     0
failures      0
schoolsup     0
famsup        0
paid          0
activities    0
nursery       0
higher        0
internet      0
romantic      0
famrel        0
freetime      0
goout         0
Dalc          0
Walc          0
health        0
absences      0
dtype: int64


## Pré-processamento dataset Student Performance

In [12]:
# creating classification targets
y_final = np.where(y['G3'] >= 10, 1, 0)  # Convertendo a nota final em uma classificação binária (aprovado/reprovado). Usando 10 como o limite de aprovação.
# 1 representa aprovado e 0 representa reprovado.
# Verificando a distribuição das classes
print("\nDistribuição das classes (Aprovado/Reprovado):")
print(pd.Series(y_final).value_counts()) # Contando as ocorrências de cada classe. usando pd.Series para criar uma série pandas a partir do array numpy





Distribuição das classes (Aprovado/Reprovado):
1    549
0    100
Name: count, dtype: int64


In [13]:
# The dataset seem unbalanced, so it has to be balanced

# tranforming text features into numerical features
X_final = pd.get_dummies(X, drop_first=True)  # Convertendo variáveis categóricas em variáveis dummy # drop_first=True evita a armadilha da variável fictícia
# Verificando as dimensões dos dados após a codificação
print("\nDimensões dos dados após a codificação:") 
print("Formato das features (X) antes do processamento:", X.shape)
print("Formato das features (X) após o processamento:", X_final.shape)
# Verificando as primeiras linhas dos dados codificados
print("\nPrimeiras linhas dos dados codificados:")
print(X_final.head())
print(X_final.dtypes)




Dimensões dos dados após a codificação:
Formato das features (X) antes do processamento: (649, 30)
Formato das features (X) após o processamento: (649, 39)

Primeiras linhas dos dados codificados:
   age  Medu  Fedu  traveltime  studytime  failures  famrel  freetime  goout  \
0   18     4     4           2          2         0       4         3      4   
1   17     1     1           1          2         0       5         3      3   
2   15     1     1           1          2         0       4         3      2   
3   15     4     2           1          3         0       3         2      2   
4   16     3     3           1          2         0       4         3      2   

   Dalc  ...  guardian_mother  guardian_other  schoolsup_yes  famsup_yes  \
0     1  ...             True           False           True       False   
1     1  ...            False           False          False        True   
2     2  ...             True           False           True       False   
3     1  ...     

## Spliting data into training and testing sets

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.25, random_state=42, stratify=y_final)  # Stratify -> mantém a proporção das classes no split

print(f"\nDimensões do conjunto de treino: {X_train.shape}")
print(f"Dimensões do conjunto de teste: {X_test.shape}")


Dimensões do conjunto de treino: (486, 39)
Dimensões do conjunto de teste: (163, 39)


## Normalizing 

In [15]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  
X_test_scaled = scaler.transform(X_test)  

## applying PCA and smote

In [16]:
# Using SMOTE to balance the dataset
from imblearn.over_sampling import SMOTE

N_COMPONENTES_SP = 5
pca_sp = PCA(n_components=N_COMPONENTES_SP)

X_train_pca_sp = pca_sp.fit_transform(X_train_scaled)
X_test_pca_sp = pca_sp.transform(X_test_scaled)

print(f"PCA aplicado. Dimensões reduzidas para {N_COMPONENTES_SP} componentes.")

# applying smote on the pca-transformed data
smote_pca = SMOTE(random_state=42)
X_train_resampled_pca, y_train_resampled_pca = smote_pca.fit_resample(X_train_pca_sp, y_train)

# Verificando o balanceamento DEPOIS do SMOTE
print("\nDistribuição de classes no treino DEPOIS do SMOTE:")
print(pd.Series(y_train_resampled_pca).value_counts())


PCA aplicado. Dimensões reduzidas para 5 componentes.

Distribuição de classes no treino DEPOIS do SMOTE:
1    411
0    411
Name: count, dtype: int64


## Training the dataset

In [17]:
# training the MLP model with the balanced dataset
mlp_student = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
mlp_student.fit(X_train_resampled_pca, y_train_resampled_pca)  # Treinando o modelo com os dados de treino balanceados

print("\nMLP com pca treinado com sucesso no dataset de students")

# predicting the test set com MLP
mlp_predictions_student = mlp_student.predict(X_test_pca_sp)
# Avaliando o modelo
print("\nRelatório de Classificação para o dataset de students:")
print(classification_report(y_test, mlp_predictions_student, target_names=['Reprovado', 'Aprovado']))



MLP com pca treinado com sucesso no dataset de students

Relatório de Classificação para o dataset de students:
              precision    recall  f1-score   support

   Reprovado       0.22      0.28      0.25        25
    Aprovado       0.86      0.82      0.84       138

    accuracy                           0.74       163
   macro avg       0.54      0.55      0.54       163
weighted avg       0.76      0.74      0.75       163

